# Raster-based market access

Following in the footsteps of the Malaria Atlas Project's <a href="https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2015_v1_0">Global Friction Surface</a>, this notebook explores a set of tools for calculating:

1. Travel time rasters
2. Drive-time polygons
3. Marketsheds


In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


ModuleNotFoundError: No module named 'infrasap'

In [2]:
tutorial_folder = os.path.join("../", "tutorial_data")

dests = os.path.join(tutorial_folder, "destinations.shp")
friction_surface = os.path.join(tutorial_folder, "global_friction_surface.tif")

inD = gpd.read_file(dests)
inR = rasterio.open(friction_surface)
frictionD = inR.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [3]:
ma.calculate_travel_time?

In [6]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, inD)
travel_costs = travel_costs.astype(inR.meta['dtype'])
with rasterio.open(os.path.join(tutorial_folder, "least_cost_travel_time.tif"), 'w', **inR.meta) as out_file:
    out_file.write_band(1, travel_costs)

In [7]:
ma.generate_feature_vectors?

In [8]:
drive_time_thresholds = [60, 120, 180, 240] # minutes
drive_vectors = ma.generate_feature_vectors(inR, mcp, inD, drive_time_thresholds)
drive_vectors.to_file(os.path.join(tutorial_folder, "drive_vectors.shp"))

14:09:06	1 of 5
14:09:06	2 of 5
14:09:07	3 of 5
14:09:07	4 of 5
14:09:08	5 of 5


In [9]:
ma.generate_market_sheds?

In [10]:
importlib.reload(ma)
outfile = os.path.join(tutorial_folder, "marketsheds.tif")
ma.generate_market_sheds(inR, inD, out_file=outfile)